In [1]:
import os
import sys
import pandas as pd
import numpy as np
import glob
import time
import gget
import scipy
from scipy.sparse import csr_matrix
import anndata as an
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import random
from importlib import reload
import warnings
import ot
from scipy.spatial.distance import pdist, squareform

import surprise as sup

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

"""WARNING: no warnings"""
warnings.filterwarnings("ignore")

# local imports
import anndata_utils as anntools

source_path = os.path.abspath("../source/")
sys.path.append(source_path)
import centrality as central
import matrix
import utils as ut
import plotting as plt2

In [2]:
resolution = 1000000
chrom = "chr2"

dpath = "/scratch/indikar_root/indikar1/shared_data/higher_order/by_chromosome/"

file_list = sorted(glob.glob(f"{dpath}*_{resolution}_{chrom}*"))
print(file_list)

population_path = file_list[0]
singlecell_path = file_list[1]

print()

print(f"{population_path=}")
print(f"{singlecell_path=}")

['/scratch/indikar_root/indikar1/shared_data/higher_order/by_chromosome/population_mESC_1000000_chr2.h5ad', '/scratch/indikar_root/indikar1/shared_data/higher_order/by_chromosome/singlecell_mESC_1000000_chr2.h5ad']

population_path='/scratch/indikar_root/indikar1/shared_data/higher_order/by_chromosome/population_mESC_1000000_chr2.h5ad'
singlecell_path='/scratch/indikar_root/indikar1/shared_data/higher_order/by_chromosome/singlecell_mESC_1000000_chr2.h5ad'


# load population

In [ ]:
start_time = time.time()  # Record the start time
adata = sc.read_h5ad(population_path)
end_time = time.time()  # Record the end time
print(f"Time taken to read the file: {end_time - start_time:.2f} seconds")
sc.logging.print_memory_usage()
adata

# load single-cell

In [ ]:
start_time = time.time()  # Record the start time
bdata = sc.read_h5ad(singlecell_path)
end_time = time.time()  # Record the end time
print(f"Time taken to read the file: {end_time - start_time:.2f} seconds")
sc.logging.print_memory_usage()
bdata

# QC

In [ ]:
def find_outliers_iqr(df_column):
  """
  Identifies outliers in a pandas DataFrame column using the IQR method.

  Args:
    df_column: A pandas Series representing the column to analyze.

  Returns:
    A boolean mask with True for outliers and False otherwise.
  """
  Q1 = df_column.quantile(0.15)
  Q3 = df_column.quantile(0.85)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  return (df_column < lower_bound) | (df_column > upper_bound)

adata.obs['degree_outlier'] = find_outliers_iqr(adata.obs['chrom_degree'])

adata.obs[adata.obs['degree_outlier']][['chrom_bin', 'chrom_degree', 'degree_outlier']].head()

In [ ]:
# remove outliers
remove_bins = adata.obs[adata.obs['degree_outlier']].index.to_list()
print(f"Removing top {len(remove_bins)} outlier loci: ")
print(remove_bins)

adata = adata[~adata.obs_names.isin(remove_bins), :].copy()
bdata = bdata[~bdata.obs_names.isin(remove_bins), :].copy()

print('done!')

# Clique-expand

In [ ]:
matrix.expand_and_normalize_anndata(adata, oe_kr=True)
print()
matrix.expand_and_normalize_anndata(bdata, oe_kr=True)

adata

# Centrality measures

In [ ]:
ce_centralities = {
    'ce_eigenvector_centrality' : {
        'function' : nx.eigenvector_centrality,
        'weight' : True
    },
    'ce_betweenness_centrality' : {
        'function' : nx.betweenness_centrality,
        'weight' : True
    },
    'ce_pagerank' : {
        'function' : nx.pagerank,
        'weight' : True
    },
}

obsm_key = 'A_oe'
A = adata.obsm[obsm_key].copy()
# A = A.mask(np.eye(A.shape[0], dtype=bool), 0)

G = nx.from_pandas_adjacency(A)
print(G)

for label, d in ce_centralities.items():
    if d['weight']:
        centrality = d['function'](G, weight='weight')
    else:
        centrality = d['function'](G)
        
    adata.obs[label] = adata.obs.index.map(centrality)
    adata.obs[label] = ut.min_max(adata.obs[label])

adata.obs[list(ce_centralities.keys())].head()

# higher-order centralities

In [ ]:
def balance_incidence_matrix(matrix, reg=0.85):
    """Balances a sparse matrix to be doubly stochastic using Sinkhorn-Knopp.

    Args:
        matrix: A scipy.sparse matrix.
        reg: Regularization parameter for Sinkhorn-Knopp algorithm.

    Returns:
        A balanced sparse matrix.
    """
    start_time = time.time()
    matrix = matrix.toarray()  # Convert to dense array for POT
    a = np.ones(matrix.shape[0]) / matrix.shape[0]  # Uniform row distribution
    b = np.ones(matrix.shape[1]) / matrix.shape[1]  # Uniform column distribution
    balanced_matrix = ot.sinkhorn(a, b, matrix, reg)  # Use input regularization parameter
    end_time = time.time()
    print(f"Balancing matrix took: {end_time - start_time:.2f} seconds")
    return csr_matrix(balanced_matrix)  # Convert back to sparse

In [ ]:
# add the principal singular value of the incidence matrix
H = adata.to_df().copy()
print(f"Raw: {H.shape=}")
H = H.T.drop_duplicates().T
print(f"De-duped: {H.shape=}")

node_weight_attr = 'ATACSeq_1' # must be an obs column
node_weights = adata.obs.loc[H.index, node_weight_attr].values


svd = TruncatedSVD(n_components=1, n_iter=10)
adata.obs['singular_vector_1'] = ut.min_max(svd.fit_transform(H))

# hypergraph centralities
hge_functions = {
    'hge_logexp_unweighted' : {
        'function' : 'log-exp',
        'weights' : None,
    },
    'hge_logexp_degree_weighted' : {
        'function' : 'log-exp',
        'weights' : 1 / (H.sum(axis=1).values + 1),
    },
    'hge_logexp_RNA_weighted' : {
        'function' : 'log-exp',
        'weights' : 1 / (adata.obs.loc[H.index, 'RNA_2'].values + 1)
    },
    'hge_logexp_ATAC_weighted' : {
        'function' : 'log-exp',
        'weights' : 1 / (adata.obs.loc[H.index, 'ATACSeq_1'].values + 1)
    },
}


hge_centralities = []

for label, d in hge_functions.items():
    start_time = time.time()  # Record start time
    node, edge = central.nonlinear_eigenvector_centrality(
        H,
        function=d['function'],
        node_weights=d['weights'],
    )

    hge_centralities.append(label)
    adata.obs[label] = ut.min_max(node)

    end_time = time.time()  # Record end time
    print(f"{label} calculation took: {end_time - start_time:.2f} seconds")

adata

In [ ]:
# umap_features = [    
#     'ATACSeq_3',
#     'CTCF',
#     'H3K27ac',
#     'H3K27me3', 
#     'RNA_2', 
#     'ce_eigenvector_centrality',
#     'singular_vector_1',
#     'hge_logexp_unweighted',
#     'hge_logexp_RNA_weighted',
#     'hge_logexp_ATAC_weighted',
# ]

# adata.obsm['X_features'] = adata.obs[umap_features].copy()

# sc.pp.neighbors(
#     adata, 
#     use_rep='X_features',
#     n_neighbors=5,
# )

# sc.tl.umap(
#     adata,
# )

# adata.obs['UMAP 1'] = adata.obsm['X_umap'][:, 0]
# adata.obs['UMAP 2'] = adata.obsm['X_umap'][:, 1]

# plt.rcParams['figure.dpi'] = 200
# plt.rcParams['figure.figsize'] = 3, 3

# sns.scatterplot(
#     data=adata.obs,
#     x='UMAP 1',
#     y='UMAP 2',
#     ec='none',
# )

# plt.yticks([])
# plt.xticks([])
# sns.despine()

In [ ]:
# plt.rcParams['figure.dpi'] = 200
# plt.rcParams['figure.figsize'] = 4, 4

# cmap = 'hot_r'
# colorbars = False

# for feature in umap_features:
#     sns.scatterplot(
#         data=adata.obs,
#         x='UMAP 1',
#         y='UMAP 2',
#         ec='k',
#         s=35,
#         hue=feature,
#         palette=cmap,
#         legend=False,
#     )

#     plt.yticks([])
#     plt.xticks([])
#     plt.title(feature)
#     sns.despine()
#     plt.show()
    
#     if colorbars:
#         plt2.make_colorbar(cmap=cmap, tick_labels=['Low', 'High'])
#         plt.show()

#     # break

# Extract the core from population

In [ ]:
core_score = 'hge_logexp_RNA_weighted'
core_threshold_quantile = 0.75
order_threshold = 2

vector = adata.obs[core_score].values
threshold = np.quantile(vector, core_threshold_quantile)
core_nodes = adata.obs[adata.obs[core_score] > threshold].index.to_list()

# extract the core from population
core = adata[core_nodes, :].copy()
core = core[:, core.X.sum(axis=0) > order_threshold].copy()

H_core = core.to_df()
print(f"{H_core.shape=}")
H_core.columns = [f"core_{x}" for x in H_core.columns]
H_core.head()

# create a set of single-cell incidence matrices

In [ ]:
incidence_matrices = {}
cell_ids = bdata.var['basename'].unique()
num_cells = len(cell_ids)
total_time = 0.0  # Initialize cumulative time

for i, cell_id in enumerate(cell_ids):
    start_time = time.time()

    # extract the single-cell
    sc_data = bdata[:, bdata.var['basename'] == cell_id].copy()
    H_o = sc_data.to_df()
    H_o = H_o.T.drop_duplicates().T  # Transpose, drop duplicates, transpose back
    H_o.columns = [f"{cell_id}_{x}" for x in H_o.columns]

    incidence_matrices[cell_id] = H_o

    # Update timing information
    elapsed_time = time.time() - start_time
    total_time += elapsed_time

    # Periodic status updates
    if (i+1) % 50 == 0:
        print(f"Processed {i+1}/{num_cells} cells. "
              f"Time for last cell: {elapsed_time:.2f} seconds, "
              f"Cumulative time: {total_time:.2f} seconds")

print("Finished processing all cells.")

In [ ]:
break

# Imputation

In [ ]:
core_dict = adata.obs[core_score].to_dict()
n_levels = 5
preds = {}

start_time = time.time()  # Start the overall timer
cumulative_time = 0  # Initialize cumulative time

n_cells = 2
cell_counter = -1

for i, (cell_id, H) in enumerate(incidence_matrices.items()):
    cell_counter += 1
    if cell_counter == n_cells:
        break
    loop_start_time = time.time()  # Start the loop timer
    print(f"Processing cell {i+1}/{len(incidence_matrices)} (cell_id: {cell_id})")

    # add the core hyperedges
    add_to = adata.to_df().copy()
    add_to = add_to.T.drop_duplicates().T
    H_imp = pd.concat([H, add_to], axis=1, ignore_index=False)
    H_imp = H_imp.fillna(0.0)
    print(f"{H_imp.shape=}")

    # reshape 
    H_imp = H_imp.reset_index(drop=False)
    H_imp = pd.melt(H_imp, id_vars='bin_name')
    
    H_imp['core_score'] = H_imp['bin_name'].map(core_dict)
    H_imp['weighted_value'] = H_imp['core_score'] * H_imp['value']
    H_imp['discretized_value'] = (H_imp['weighted_value'] * n_levels).astype(int)
    H_imp = H_imp.rename(columns={
        'variable': 'userID',
        'bin_name': 'itemID',
        'discretized_value' : 'rating',
    })

    reader = sup.Reader(rating_scale=(0, n_levels))
    data = sup.Dataset.load_from_df(H_imp[["userID", "itemID", "rating"]], reader)
    trainset = data.build_full_trainset()


    sim_options = {"name": "pearson_baseline"}
    
    algo = sup.SVDpp(sim_options=sim_options)
    algo.fit(trainset)

    """COULD MAKE UP NEW HYPEREDGES """
    H_pred = []
    for read_name in H.columns.to_list():
        for bin_name in H.index.to_list():
            pred = algo.predict(uid=read_name, iid=bin_name)
            row = {
                'read_name': read_name,
                'bin_name': bin_name,
                'value': pred.est,
            }
            H_pred.append(row)

    H_pred = pd.DataFrame(H_pred)
    H_pred = pd.pivot_table(
        H_pred,
        index='bin_name',
        columns='read_name',
        values='value',
        fill_value=0.0,
    )

    preds[cell_id] = H_pred

    loop_end_time = time.time()  # End the loop timer
    cell_time = loop_end_time - loop_start_time
    cumulative_time += cell_time  # Update cumulative time
    print(f"Cell processed in {cell_time:.2f} seconds (cumulative: {cumulative_time:.2f} seconds)")
    break

end_time = time.time()  # End the overall timer
print(f"Total processing time: {end_time - start_time:.2f} seconds")

In [ ]:
cell_id = 'o2b67'
fig, axs = plt.subplots(1, 2, sharey=True)

axs[0].imshow(incidence_matrices[cell_id])
axs[1].imshow(preds[cell_id])
plt.tight_layout()

In [ ]:
break

In [ ]:
# core_dict = adata.obs[core_score].to_dict()

# preds = {}


# for cell_id, H in incidence_matrices.items():
    
#     read_names = H.columns.to_list()
#     bin_names = H.index.to_list()
#     print(f"{H.shape=}")

#     # append the  core
#     H = pd.concat([H, H_core], axis=1, ignore_index=False)
#     H = H.fillna(0.0)
#     print(f"{H.shape=}")

#     # develop the dataset from the combined data
#     H = H.reset_index(drop=False)
#     H = pd.melt(H, id_vars='bin_name')
#     H['core_score'] = H['bin_name'].map(core_dict)
#     H['rating'] = H['core_score'] * H['value']
#     H = H.rename(columns={
#         'variable' : 'userID',
#         'bin_name' : 'itemID',
#     })

#     # build the data set
#     reader = Reader(rating_scale=(0, 1))
#     data = Dataset.load_from_df(H[["userID", "itemID", "rating"]], reader)
#     trainset = data.build_full_trainset()

#     # Build an algorithm, and train it.
#     algo = SVD()
#     algo.fit(trainset)

#     # predict only on sc hyperedges
#     H_pred = []
#     for read_name in read_names:
#         for bin_name in bin_names:
#             pred = algo.predict(uid=read_name, iid=bin_name)
#             row = {
#                 'read_name' : read_name,
#                 'bin_name' : bin_name,
#                 'value' : pred.est,
#             }
#             H_pred.append(row)

#     H_pred = pd.DataFrame(H_pred)
#     H_pred = pd.pivot_table(
#         H_pred,
#         index='bin_name',
#         columns='read_name',
#         values='value',
#         fill_value=0.0,
#     )

#     print(f"{H_pred.shape=}")

#     preds[cell_id] = H_pred



In [ ]:
dir(pred)

In [ ]:
break